In [6]:
import pandas as pd
from datetime import date
pd.set_option('display.max_columns', None)

In [7]:
link_author_affiliation = catalog.load('intermediate/openalex/link_author_affiliation#parquet')

[08/30/24 13:01:06] INFO     Loading data from                                                  ]8;id=630892;file:///home/pablo/dev/CIC/dw-cic/kedro-unlp/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=193614;file:///home/pablo/dev/CIC/dw-cic/kedro-unlp/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             intermediate/openalex/link_author_affiliation#parquet                                 
                             (ParquetDataset)...                                                                   

In [8]:
link_author_affiliation.sample(2)

,author.id,author.orcid,author.display_name,institution.country_code,institution.display_name,institution.id,institution.lineage,institution.ror,institution.type
913,https://openalex.org/A5028696402,None,Juan Manuel Moro,AR,Universidad Nacional del Sur,https://openalex.org/I56858762,[https://openalex.org/I56858762],https://ror.org/028crwz56,education
265,https://openalex.org/A5032933290,https://orcid.org/0000-0002-8514-0584,Romi L. Burks,US,Southwestern University,https://openalex.org/I8616870,[https://openalex.org/I8616870],https://ror.org/05gj63w50,education


In [9]:
def stage_institution2institution_openalex(link_author_affiliation):
    link_institution_parent = link_author_affiliation[['institution.id','institution.lineage']].drop_duplicates('institution.id')
    link_institution_parent = link_institution_parent.explode('institution.lineage')
    link_institution_parent = link_institution_parent.rename(columns={'institution.lineage':'parent.id'})
    filter = link_institution_parent['institution.id'] != link_institution_parent['parent.id']
    link_institution_parent[filter]
    link_institution_parent['load_datetime'] = date.today()
    link_institution_parent.columns = link_institution_parent.columns.str.replace('.', '_')

    return link_institution_parent

In [10]:
link_institution_parent = stage_institution2institution_openalex(link_author_affiliation)
link_institution_parent

,institution_id,parent_id,load_datetime
0,https://openalex.org/I128590013,https://openalex.org/I128590013,2024-08-30
1,https://openalex.org/I151201029,https://openalex.org/I151201029,2024-08-30
1,https://openalex.org/I151201029,https://openalex.org/I4210123736,2024-08-30
1,https://openalex.org/I151201029,https://openalex.org/I4387155568,2024-08-30
2,https://openalex.org/I228664971,https://openalex.org/I228664971,2024-08-30
...,...,...,...
1639,https://openalex.org/I4210099012,https://openalex.org/I4210099012,2024-08-30
1639,https://openalex.org/I4210099012,https://openalex.org/I4210127390,2024-08-30
1639,https://openalex.org/I4210099012,https://openalex.org/I4210138501,2024-08-30
1639,https://openalex.org/I4210099012,https://openalex.org/I4210151987,2024-08-30
